In [1]:

import pandas as pd
import scipy.integrate as spi
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import matplotlib as mlb
import math
from scipy.stats import chisquare


In [ ]:
beta=1.0

gamma=1/10.0
sigma=1/12.0
mu=5e-4
N0=2618.0
mu_d=1/20
### You may want to try with popylation size of 50 (small) to see the events
### In this case uncomment the next line
#N0=50.0
ND=MaxTime=1*365.0
mort=334/2618
# X0=pl.floor(gamma*N0/beta)
# Y0=pl.ceil(mu*N0/gamma)
# Z0=N0-X0-Y0
W0=N0-1
X0=1
Y0=0
Z0=0
INPUT = np.array((W0,X0,Y0,Z0))

timestep=0.0

def stoc_eqs(INP,ts): 
	V = INP
	Rate=np.zeros((5))
	Change=np.zeros((5,4))
	N=np.sum(V[range(4)])
	Rate[0] = beta*V[0]*V[2]/N; Change[0,:]=([-1, +1, 0, 0]);
	Rate[1] = sigma*V[1];  Change[1,:]=([0, -1, +1, 0]);
	Rate[2] = gamma*V[2];  Change[2,:]=([0, 0,-1, +1]);
# 	Rate[3] = mu*N;  Change[3,:]=([+1, 0, 0, 0]);
# 	Rate[4] = mu*V[0];  Change[4,:]=([-1, 0, 0, 0]);
# 	Rate[5] = mu*V[1];  Change[5,:]=([0, -1, 0, 0]);
# 	Rate[6] = mu*V[2];  Change[6,:]=([0, 0, -1, 0]);
# 	Rate[7] = mu*V[3];  Change[7,:]=([0, 0, 0, -1]);
	Rate[4] = mort*V[2];  Change[4,:]=([0, 0, -1, 0]);
	R1=pl.rand();
	R2=pl.rand();
	ts = -np.log(R2)/(np.sum(Rate));
	m=min(pl.find(pl.cumsum(Rate)>=R1*pl.sum(Rate)));
	V[range(4)]=V[range(4)]+Change[m,:]
	return [V,ts]

def Stoch_Iteration(INPUT):
	lop=0
	ts=0
	T=[0]
	S=[0]
	E=[0]
	I=[0]
	R=[0]
	while T[lop] < ND:
		lop=lop+1
		T.append(T[lop-1]+ts)
		S.append(INPUT[0])
		E.append(INPUT[1])
		I.append(INPUT[2])
		R.append(INPUT[3])
		[res,ts] = stoc_eqs(INPUT,ts)
		lop=lop+1
		T.append(T[lop-1])
		S.append(INPUT[0])
		E.append(INPUT[1])
		I.append(INPUT[2])
		R.append(INPUT[3])
	return [T,S,E,I,R]

[T,S,E,I,R]=Stoch_Iteration(INPUT)

tT=np.array(T)[1:,]/365.
tS=np.array(S)[1:,]
tE=np.array(E)[1:,]
tI=np.array(I)[1:,]
tR=np.array(R)[1:,]
pl.figure(figsize=(15,10))
pl.subplot(411)
pl.plot(tT, tS, 'g')
#pl.xlabel ('Time (years)')
pl.ylabel ('Susceptible')
pl.subplot(412)
pl.plot(tT, tE, 'g')
#pl.xlabel ('Time (years)')
pl.ylabel ('Exposed')
pl.subplot(413)
pl.plot(tT, tI, 'r')
#pl.xlabel ('Time (years)')
pl.ylabel ('Infectious')
pl.subplot(414)
pl.plot(tT, tR, 'k')
pl.xlabel ('Time (years)')
pl.ylabel ('Recovered')
pl.show()